# IBM 1 and 2 models Code  Implementation

f is the source language, e the target language. In this code, we use the convention that we represent our data in the following way:
(source, target, alignment)


In [1]:
%load_ext autoreload
%autoreload 2

import random
import numpy as np
import math
import time
from aer import *
import codecs
from abc import ABC
import pickle as cPickle
import os

## Constants.py

In [2]:
# READING

TRAINING_DIRECTORY = 'data/training/'
TRAINING_ENGLISH_FILENAME = 'hansards.36.2.e'
TRAINING_FRENCH_FILENAME = 'hansards.36.2.f'
TRAINING_PAIR_FILENAME = 'training_pairs'

VALIDATION_DIRECTORY = 'data/validation/'
VALIDATION_ENGLISH_FILENAME = 'dev.e'
VALIDATION_FRENCH_FILENAME = 'dev.f'
VALIDATION_PAIR_FILENAME = 'validation_pairs'
VALIDATION_ALIGNMENTS_FILENAME = 'dev.wa.nonullalign'

TEST_DIRECTORY = 'data/testing/'
TEST_ENGLISH_FILENAME = 'test/test.e'
TEST_FRENCH_FILENAME = 'test/test.f'
TEST_ALIGNMENTS_FILENAME = 'answers/test.wa.nonullalign'

# WRITING

IBM1_OUTPUT_DIR = "models/ibm1/"
IBM1B_OUTPUT_DIR = "models/ibm1_b/"
IBM2_OUTPUT_DIR = "models/ibm2/"

TEST_ALIGNMENTS_OUTPUT_IBM1 = "ibm1.mle.naacl"
TEST_ALIGNMENTS_OUTPUT_IBM1B = "ibm1.vb.naacl"
TEST_ALIGNMENTS_OUTPUT_IBM2 = "ibm2.mle.naacl"



## Utils.py

In [3]:
def get_AER(predictions, test):
    metric = AERSufficientStatistics()

    for gold, pred in zip(test, predictions):
        metric.update(sure=gold[0], probable=gold[1], predicted=pred)

    return metric.aer()

# todo add test
def read_data(english_file, french_file, save = 0, path = ""):
    with open(english_file) as f:
        sentences_english = f.read().splitlines()
    with open(french_file) as f:
        sentences_french = f.read().splitlines()

    paired = []
    for i, sentence_english in enumerate(sentences_english):
        paired.append([("null " + sentence_english).split(" ")[0:-1],
                       sentences_french[i].split(" ")[0:-1]])
    
    if save:
        cPickle.dump(paired, open(str(path), "wb"))

    return paired

def get_validation_alignments(path = VALIDATION_DIRECTORY + VALIDATION_ALIGNMENTS_FILENAME):
    validation_alignments = read_naacl_alignments(path)
    return validation_alignments

def get_vocabulary_size( data):
    frenchWords = []
    for pair in (data):
        for word in pair[1]:
            frenchWords.append(word)
    return len(frenchWords)

## IBM_base.py


In [4]:
class IBM_base(ABC):
    UNIFORM_INIT = "uniform"
    RANDOM_INIT = "random"

    init_method = None
    t = []
    train_data = []
    val_data = []
    val_alignments = []

    def train(self):
        pass

    def get_alignments(self):
        pass

    def evaluate(self):
        pass

    def set_t(self, t):
        self.t = t
        
    def empty_init(self, train_data):
        t = {}
        for i, pair in enumerate(train_data):
            for english_word in pair[0]:
                if english_word not in t:
                    t[english_word] = {}
                for french_word in pair[1]:
                    t[english_word][french_word] = 0
        #print(t["36"])
        return t

    def uniform_init(self, t):
        # Uniform init of the translation probabilities
        new_t = {}

        for key in t:
            new_t[key] = {}
            vocab_size = len((t[key].keys()))

            for sec_key in t[key]:
                new_t[key][sec_key] = 1.0 / vocab_size
        
        self.t = new_t
        #print(self.t["36"])
        return

    def random_init(self, t):
        # Random init of the translation probabilities
        new_t = {}
        
        for key in t:
            new_t[key] = {}

            for sec_key in t[key]:
                new_t[key][sec_key] = random.random()

            normalizer = sum(new_t[key].values())
            new_t[key] = {k: v / normalizer for k, v in new_t[key].items()}
        #print(self.t["36"])
        self.t = new_t
            
    def init_empty_english_counts(self, t):
        empty_counts = {}
        english_empty_counts = {}

        for key in t:
            empty_counts[key] = {}
            english_empty_counts[key] = 0.0
            for sec_key in t[key]:
                empty_counts[key][sec_key] = 0.0

        return english_empty_counts, empty_counts

## IBM1

In [5]:
class IBM1(IBM_base):

    def __init__(self, train_data, val_data, val_alignments, init_method="uniform"):
        self.init_method = init_method
        self.train_data = train_data
        self.val_data = val_data
        self.val_alignments = val_alignments
        
        t = self.empty_init(train_data)

        if self.init_method == self.UNIFORM_INIT:
            self.uniform_init(t)
        elif self.init_method == self.RANDOM_INIT:
            self.random_init(t)
        else:
            print("Invalid init method, defaulting to uniform")
            self.init_method = self.UNIFORM_INIT
            self.uniform_init(t)

    def get_alignments(self, val_pairs, t):
        """Get the predicted alignments on sentence pairs from a trained ibm model 1 or 2"""
        alignments = []
        for k, val_pair in enumerate(val_pairs):
            alignments.append(set())
            for j, french_word in enumerate(val_pair[1]):
                max_prob = 0.0
                alignment = 0

                for i, english_word in enumerate(val_pair[0]):
                    if english_word in t:
                        if french_word in t[english_word]:
                            align_prob = t[english_word][french_word]

                    if align_prob > max_prob:
                        max_prob = align_prob
                        alignment = i
                if alignment is not 0:
                    alignments[k].add((alignment, j + 1))

        return alignments

    def evaluate(self, t):
        predictions = self.get_alignments(self.val_data, t)

        aer = get_AER(predictions, self.val_alignments)

        return aer

    def train(self, treshold, val_pairs = False, val_alignments = False, aer_epochs_treshold = 5):
        log_likelihood = []
        aers = []
        
        t = self.t
        best_t = t

        number_of_sentences = len(self.train_data)
        min_aer = float('inf')
        epoch = 0

        english_empty_counts, empty_counts = self.init_empty_english_counts(t)
        # computing empty counts
        empty_counts = {}
        english_empty_counts = {}
        for key in t:
            empty_counts[key] = {}
            english_empty_counts[key] = 0.0
            for secKey in t[key]:
                empty_counts[key][secKey] = 0.0


        converged = False

        while not converged:
            start = time.time()
            log_like = 0
            epoch += 1

            counts = empty_counts
            english_counts = english_empty_counts

            # Expectation - step
            print("E step")
            for pair in self.train_data:

                for j, french_word in enumerate(pair[1]):
                    normalizer = 0.0
                    for i, english_word in enumerate(pair[0]):
                        normalizer += t[english_word][french_word]

                    log_like += np.log(normalizer)

                    for i, english_word in enumerate(pair[0]):
                        delta = t[english_word][french_word] / normalizer
                        counts[english_word][french_word] += delta
                        english_counts[english_word] += delta

            print("M step")
            for english_word in t:
                for french_word in t[english_word]:
                    t[english_word][french_word] = counts[english_word][french_word] / english_counts[english_word]

            log_likelihood.append(log_like / number_of_sentences)

            aer = self.evaluate(t)
            aers.append(aer)

            if aer < min_aer:
                min_aer = aer
                best_t = t

            if epoch > aer_epochs_treshold:
                if len(log_likelihood) > 1:
                    diff = log_likelihood[-1] - log_likelihood[-2]
                    if diff < treshold:
                        converged = True

            end = time.time()
            print("epoch: ", epoch, " aer: ", aer, " loglikelihood: ", log_likelihood[-1], " time: ", end - start)

        self.t = t

        return t, best_t

## IBM2


In [6]:
class IBM2(IBM_base):
    IBM1_INIT = "ibm1_init"

    a = []

    def __init__(self, train_data, val_data, val_alignments, a=dict, init_method="uniform", model_path=""):
        self.init_method = init_method
        self.train_data = train_data
        self.val_data = val_data
        self.val_alignments = val_alignments

        t = self.empty_init(train_data)

        if init_method == self.UNIFORM_INIT:
            self.uniform_init(t)
        elif init_method == self.RANDOM_INIT:
            self.random_init(t)
        elif init_method == self.IBM1_INIT:
            if model_path:
                self.model_init(model_path)
            else:
                print("No path provided for IBM1 model, defaulting to uniform init")
                self.init_method = self.UNIFORM_INIT
                self.uniform_init(t)
        else:
            print("Invalid init method, defaulting to uniform init")
            self.init_method = self.UNIFORM_INIT
            self.uniform_init(t)

    def set_a(self, a):
        self.a = a

    def init_a(self):
        ''' Initialize voger count parameter vector'''
        a_counts = {}
        a = {}

        for train_pair in self.train_data:
            I = len(train_pair[0])
            J = len(train_pair[1])

            for i, eng_word in enumerate(train_pair[0]):
                for j, french_word in enumerate(train_pair[1]):
                    a_counts[self.a_index(i, j, I, J)] = 0.0

        if self.init_method == self.UNIFORM_INIT:
            length = len(a_counts.keys())
            for key in a_counts:
                a[key] = 1.0 / length
        elif self.init_method == self.RANDOM_INIT:
            for key in a_counts:
                a[key] = random.random()

            normalizer = sum(a.values())
            a = {k: v / normalizer for k, v in a.items()}
        elif self.init_method == self.IBM1_INIT:
            a = self.a
            a_counts = {k: 0.0 for k in a.keys()}

        return a, a_counts


    def a_index(self, i, j, I, J):
        # get a index count
        return math.floor(i - (j + 1.0) * I / J)


    def get_alignments(self, val_pairs, t, a=dict):
        """Get the predicted alignments on sentence pairs from a trained ibm model 1 or 2"""
        alignments = []
        for k, val_pair in enumerate(val_pairs):
            alignments.append(set())
            I = len(val_pair[0])
            J = len(val_pair[1])

            for j, french_word in enumerate(val_pair[1]):
                max_prob = 0.0
                align_prob = float('-inf')
                alignment = 0

                for i, english_word in enumerate(val_pair[0]):
                    if english_word in t:
                        if french_word in t[english_word]:
                            align_prob = t[english_word][french_word] * a[self.a_index(i, j, I, J)]

                    if align_prob > max_prob:
                        max_prob = align_prob
                        alignment = i
                if alignment is not 0:
                    alignments[k].add((alignment, j + 1))

        return alignments


    def evaluate(self, t, a):
        predictions = self.get_alignments(self.val_data, t, a)

        aer = get_AER(predictions, self.val_alignments)

        return aer


    def train(self, treshold, aer_epochs_treshold = 5):
        log_likelihood = []
        aers = []

        t = self.t
        best_t = self.t
        a, a_counts = self.init_a()
        best_a = a

        number_of_sentences = len(self.train_data)
        min_aer = float('inf')
        epoch = 0

        english_empty_counts, empty_counts = self.init_empty_english_counts(t)

        converged = False

        while not converged:
            start = time.time()
            log_like = 0
            epoch += 1

            counts = empty_counts
            english_counts = english_empty_counts

            # Expectation - step
            print("E step - epoch " + str(epoch))
            for train_pair in self.train_data:
                I = len(train_pair[0])
                J = len(train_pair[1])


                for j, french_word in enumerate(train_pair[1]):
                    normalizer = 0.0

                    for i, english_word in enumerate(train_pair[0]):
                        a_index = self.a_index(i, j, I, J)
                        normalizer += t[english_word][french_word] * a[a_index]

                    log_like += np.log(normalizer)

                    for i, english_word in enumerate(train_pair[0]):
                        a_index = self.a_index(i, j, I, J)
                        delta = a[a_index] * t[english_word][french_word] / normalizer

                        a_counts[a_index] += delta
                        counts[english_word][french_word] += delta
                        english_counts[english_word] += delta

            # Maximization - step
            print("M step - epoch " + str(epoch))
            for english_key in t:
                for french_key in t[english_key]:
                    t[english_key][french_key] = counts[english_key][french_key] / english_counts[english_key]

            normalizer = sum(a_counts.values())
            a = {k: v / normalizer for k, v in a_counts.items()}

            log_likelihood.append(log_like / number_of_sentences)

            aer = self.evaluate(t, a)
            aers.append(aer)

            if aer < min_aer:
                min_aer = aer
                best_a = a
                best_t = t

            if epoch > aer_epochs_treshold:
                if len(log_likelihood) > 1:
                    diff = log_likelihood[-1] - log_likelihood[-2]
                    if diff < treshold:
                        converged = True

            end = time.time()
            print("epoch: ", epoch, " aer: ", aer, " loglikelihood: ", log_likelihood[-1], " time: ", end - start)

        return t, a, best_t, best_a

## IBM Variational Bayes

# IBM Models experiments

## Read in data and define global constants

In [7]:
TRESHOLD = 1

train_pairs = read_data(TRAINING_DIRECTORY + TRAINING_ENGLISH_FILENAME, TRAINING_DIRECTORY + TRAINING_FRENCH_FILENAME)
val_pairs = read_data(VALIDATION_DIRECTORY + VALIDATION_ENGLISH_FILENAME, VALIDATION_DIRECTORY + VALIDATION_FRENCH_FILENAME)
test_pairs = read_data(TEST_DIRECTORY + TEST_ENGLISH_FILENAME, TEST_DIRECTORY + TEST_FRENCH_FILENAME)

val_alignments = get_validation_alignments()
test_alignments = get_validation_alignments(TEST_DIRECTORY + TEST_ALIGNMENTS_FILENAME)


# french_words = get_vocabulary_size(train_pairs)


## IBM1 TRAINING

In [8]:
ibm1 = IBM1(train_pairs, val_pairs, val_alignments)
t_ibm1, aer_t_ibm1 = ibm1.train(TRESHOLD)

{'36': 0.0008598452278589854, 'e': 0.0008598452278589854, 'Législature': 0.0008598452278589854, ',': 0.0008598452278589854, '2': 0.0008598452278589854, 'ième': 0.0008598452278589854, 'Session': 0.0008598452278589854, 'ouverture': 0.0008598452278589854, 'DE': 0.0008598452278589854, 'LA': 0.0008598452278589854, 'DEUXIÈME': 0.0008598452278589854, 'SESSION': 0.0008598452278589854, '36E': 0.0008598452278589854, 'LÉGISLATURE': 0.0008598452278589854, 'le': 0.0008598452278589854, 'Président': 0.0008598452278589854, 'donne': 0.0008598452278589854, 'lecture': 0.0008598452278589854, 'de': 0.0008598452278589854, 'une': 0.0008598452278589854, 'lettre': 0.0008598452278589854, 'secrétaire': 0.0008598452278589854, 'la': 0.0008598452278589854, 'Gouverneure': 0.0008598452278589854, 'générale': 0.0008598452278589854, 'annonçant': 0.0008598452278589854, 'que': 0.0008598452278589854, 'Leurs': 0.0008598452278589854, 'Excellences': 0.0008598452278589854, 'et': 0.0008598452278589854, 'John': 0.000859845227858

E step
M step
epoch:  1  aer:  0.37238095238095237  loglikelihood:  -89.90830347570255  time:  119.8903579711914
E step


KeyboardInterrupt: 

## IBM2 TRAINING

In [ ]:
ibm2 = IBM2(train_pairs, val_pairs, val_alignments)
t, a, best_t, best_a = ibm2.train(TRESHOLD)

{'36': 0.0008598452278589854, 'e': 0.0008598452278589854, 'Législature': 0.0008598452278589854, ',': 0.0008598452278589854, '2': 0.0008598452278589854, 'ième': 0.0008598452278589854, 'Session': 0.0008598452278589854, 'ouverture': 0.0008598452278589854, 'DE': 0.0008598452278589854, 'LA': 0.0008598452278589854, 'DEUXIÈME': 0.0008598452278589854, 'SESSION': 0.0008598452278589854, '36E': 0.0008598452278589854, 'LÉGISLATURE': 0.0008598452278589854, 'le': 0.0008598452278589854, 'Président': 0.0008598452278589854, 'donne': 0.0008598452278589854, 'lecture': 0.0008598452278589854, 'de': 0.0008598452278589854, 'une': 0.0008598452278589854, 'lettre': 0.0008598452278589854, 'secrétaire': 0.0008598452278589854, 'la': 0.0008598452278589854, 'Gouverneure': 0.0008598452278589854, 'générale': 0.0008598452278589854, 'annonçant': 0.0008598452278589854, 'que': 0.0008598452278589854, 'Leurs': 0.0008598452278589854, 'Excellences': 0.0008598452278589854, 'et': 0.0008598452278589854, 'John': 0.000859845227858

E step - epoch 1
M step - epoch 1
epoch:  1  aer:  0.2857142857142857  loglikelihood:  -221.70279402830207  time:  249.0057168006897
E step - epoch 2
